# 实战项目 4：多因子模型
## 说明
每个问题都包含需要实现的函数和如何实现该函数的说明。`# TODO` 注释表示需要实现的函数部分。实现函数后，请运行单元格并用我们提供的单元测试检测代码。对于每个问题，我们都在  `project_tests`  软件包中提供了一个或多个单元测试。这些单元测试不会指出你的答案是否正确，但是会提醒你重大错误。当你提交项目后，我们会检查你的答案是否正确。

## 软件包
在实现函数时，你只需使用在教室中用到的软件包，例如 [Pandas](https://pandas.pydata.org/) 和 [Numpy](http://www.numpy.org/)。我们会为你导入这些软件包。建议不要添加任何导入语句，否则打分老师可能无法运行你的代码。

我们还导入了以下其他软件包：`project_helper` 和 `project_tests`。这些是专门帮助你解决问题的自定义软件包。 `project_helper` 模块包含实用函数和图形函数。 `project_tests` 包含所有问题的单元测试。

### 安装软件包

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

### 加载软件包

In [ ]:
import cvxpy as cvx
import numpy as np
import pandas as pd
import time
import project_tests
import project_helper

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (14, 8)

## 数据包
我们将使用 Zipline 处理数据。我们为此项目创建了一个日结束数据包。请运行以下单元格，以在 zipline 中注册此数据包。

In [ ]:
import os
import project_helper
from zipline.data import bundles

os.environ['ZIPLINE_ROOT'] = os.path.join(os.getcwd(), '..', '..', 'data', 'project_4_eod')

ingest_func = bundles.csvdir.csvdir_equities(['daily'], project_helper.EOD_BUNDLE_NAME)
bundles.register(project_helper.EOD_BUNDLE_NAME, ingest_func)

print('Data Registered')

## 构建管道引擎
我们将使用 Zipline 的管道软件包访问此项目的数据。为了使用管道软件包，我们必须构建管道引擎，请运行以下单元格来构建引擎。

In [ ]:
from zipline.pipeline import Pipeline
from zipline.pipeline.factors import AverageDollarVolume
from zipline.utils.calendars import get_calendar


universe = AverageDollarVolume(window_length=120).top(500) 
trading_calendar = get_calendar('NYSE') 
bundle_data = bundles.load(project_helper.EOD_BUNDLE_NAME)
engine = project_helper.build_pipeline_engine(bundle_data, trading_calendar)

### 查看数据

构建管道引擎后，我们获取时段结束时股票池中的股票。我们将使用这些 ticker 生成风险模型的收益率数据。

In [ ]:
universe_end_date = pd.Timestamp('2016-01-05', tz='UTC')

universe_tickers = engine\
    .run_pipeline(
        Pipeline(screen=universe),
        universe_end_date,
        universe_end_date)\
    .index.get_level_values(1)\
    .values.tolist()
    
universe_tickers

## 获取收益率
构建管道后，我们来看看收益率数据。首先构建一个 DataPortal。

In [ ]:
from zipline.data.data_portal import DataPortal


data_portal = DataPortal(
    bundle_data.asset_finder,
    trading_calendar=trading_calendar,
    first_trading_day=bundle_data.equity_daily_bar_reader.first_trading_day,
    equity_minute_reader=None,
    equity_daily_reader=bundle_data.equity_daily_bar_reader,
    adjustment_reader=bundle_data.adjustment_reader)

为了便于读懂代码，我们构建了从 DataPortal 获取股价的辅助函数 `get_pricing`。

In [ ]:
def get_pricing(data_portal, trading_calendar, assets, start_date, end_date, field='close'):
    end_dt = pd.Timestamp(end_date.strftime('%Y-%m-%d'), tz='UTC', offset='C')
    start_dt = pd.Timestamp(start_date.strftime('%Y-%m-%d'), tz='UTC', offset='C')

    end_loc = trading_calendar.closes.index.get_loc(end_dt)
    start_loc = trading_calendar.closes.index.get_loc(start_dt)

    return data_portal.get_history_window(
        assets=assets,
        end_dt=end_dt,
        bar_count=end_loc - start_loc,
        frequency='1d',
        field=field,
        data_frequency='daily')

### 查看数据
我们使用 `get_pricing` 函数获取风险模型的收益率数据。对于此模型，我们将查看过去 5 年的数据。

In [ ]:
five_year_returns = \
    get_pricing(
        data_portal,
        trading_calendar,
        universe_tickers,
        universe_end_date - pd.DateOffset(years=5),
        universe_end_date)\
    .pct_change()[1:].fillna(0)

five_year_returns

# 统计学风险模型
下面开始构建风险模型。你将使用 PCA 创建一个统计学风险模型。首先构建 PCA 模型。
## 拟合 PCA
实现 `fit_pca` 以用 PCA 模型拟合收益率数据。

In [ ]:
from sklearn.decomposition import PCA


def fit_pca(returns, num_factor_exposures, svd_solver):
    """
    Fit PCA model with returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    num_factor_exposures : int
        Number of factors for PCA
    svd_solver: str
        The solver to use for the PCA model

    Returns
    -------
    pca : PCA
        Model fit to returns
    """
    #TODO: Implement function
    
    return None


project_tests.test_fit_pca(fit_pca)

### 查看数据
我们看看模型的样貌。首先查看 PCA 成分。

In [ ]:
num_factor_exposures = 20
pca = fit_pca(five_year_returns, num_factor_exposures, 'full')

pca.components_

再看看每个因子解释的方差百分比

In [ ]:
plt.bar(np.arange(num_factor_exposures), pca.explained_variance_ratio_)

可以看出前两个因子解释了大部分方差。每个因子在潜在模型中的精确定义是未知的，但是我们可以猜测可能的解释百分比。

## 因子 β
实现 `factor_betas` 以获取 PCA 模型的因子 β。

In [ ]:
def factor_betas(pca, factor_beta_indices, factor_beta_columns):
    """
    Get the factor betas from the PCA model.

    Parameters
    ----------
    pca : PCA
        Model fit to returns
    factor_beta_indices : 1 dimensional Ndarray
        Factor beta indices
    factor_beta_columns : 1 dimensional Ndarray
        Factor beta columns

    Returns
    -------
    factor_betas : DataFrame
        Factor betas
    """
    assert len(factor_beta_indices.shape) == 1
    assert len(factor_beta_columns.shape) == 1
    
    #TODO: Implement function
    
    return None


project_tests.test_factor_betas(factor_betas)

### 查看数据
下面查看此模型的因子 β。

In [ ]:
risk_model = {}
risk_model['factor_betas'] = factor_betas(pca, five_year_returns.columns.values, np.arange(num_factor_exposures))

risk_model['factor_betas']

## 因子收益率
实现 `factor_returns` 以使用收益率数据获取 PCA 模型的因子收益率。

In [ ]:
def factor_returns(pca, returns, factor_return_indices, factor_return_columns):
    """
    Get the factor returns from the PCA model.

    Parameters
    ----------
    pca : PCA
        Model fit to returns
    returns : DataFrame
        Returns for each ticker and date
    factor_return_indices : 1 dimensional Ndarray
        Factor return indices
    factor_return_columns : 1 dimensional Ndarray
        Factor return columns

    Returns
    -------
    factor_returns : DataFrame
        Factor returns
    """
    assert len(factor_return_indices.shape) == 1
    assert len(factor_return_columns.shape) == 1
    
    #TODO: Implement function
    
    return None


project_tests.test_factor_returns(factor_returns)

### 查看数据
下面看看这些因子收益率随时间推移的变化。

In [ ]:
risk_model['factor_returns'] = factor_returns(
    pca,
    five_year_returns,
    five_year_returns.index,
    np.arange(num_factor_exposures))

risk_model['factor_returns'].cumsum().plot(legend=None)

## 因子协方差矩阵
实现 `factor_cov_matrix` 以获取因子协方差矩阵

In [ ]:
def factor_cov_matrix(factor_returns, ann_factor):
    """
    Get the factor covariance matrix

    Parameters
    ----------
    factor_returns : DataFrame
        Factor returns
    ann_factor : int
        Annualization factor

    Returns
    -------
    factor_cov_matrix : 2 dimensional Ndarray
        Factor covariance matrix
    """
    
    #TODO: Implement function
    
    return None


project_tests.test_factor_cov_matrix(factor_cov_matrix)

### 查看数据

In [ ]:
ann_factor = 252
risk_model['factor_cov_matrix'] = factor_cov_matrix(risk_model['factor_returns'], ann_factor)

risk_model['factor_cov_matrix']

## 特质方差矩阵
实现 `idiosyncratic_var_matrix` 以获取特质方差矩阵。

In [ ]:
def idiosyncratic_var_matrix(returns, factor_returns, factor_betas, ann_factor):
    """
    Get the idiosyncratic variance matrix

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    factor_returns : DataFrame
        Factor returns
    factor_betas : DataFrame
        Factor betas
    ann_factor : int
        Annualization factor

    Returns
    -------
    idiosyncratic_var_matrix : DataFrame
        Idiosyncratic variance matrix
    """
    
    #TODO: Implement function
    
    return None


project_tests.test_idiosyncratic_var_matrix(idiosyncratic_var_matrix)

### 查看数据

In [ ]:
risk_model['idiosyncratic_var_matrix'] = idiosyncratic_var_matrix(five_year_returns, risk_model['factor_returns'], risk_model['factor_betas'], ann_factor)

risk_model['idiosyncratic_var_matrix']

## 特质方差向量
实现 `idiosyncratic_var_vector` 以获取特质方差向量。

In [ ]:
def idiosyncratic_var_vector(returns, idiosyncratic_var_matrix):
    """
    Get the idiosyncratic variance vector

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    idiosyncratic_var_matrix : DataFrame
        Idiosyncratic variance matrix

    Returns
    -------
    idiosyncratic_var_vector : DataFrame
        Idiosyncratic variance Vector
    """
    
    #TODO: Implement function
    
    return None


project_tests.test_idiosyncratic_var_vector(idiosyncratic_var_vector)

### 查看数据

In [ ]:
risk_model['idiosyncratic_var_vector'] = idiosyncratic_var_vector(five_year_returns, risk_model['idiosyncratic_var_matrix'])

risk_model['idiosyncratic_var_vector']

## 使用风险模型进行预测
利用我们在风险模型中计算的数据，实现 `predict_portfolio_risk` 以根据公式 $ \sqrt{X^{T}(BFB^{T} + S)X} $ 预测投资组合风险，其中：
- $ X $ 是投资组合权重
- $ B $ 是因子 β
- $ F $ 是因子协方差矩阵
- $ S $ 是特质方差矩阵

In [ ]:
def predict_portfolio_risk(factor_betas, factor_cov_matrix, idiosyncratic_var_matrix, weights):
    """
    Get the predicted portfolio risk
    
    Formula for predicted portfolio risk is sqrt(X.T(BFB.T + S)X) where:
      X is the portfolio weights
      B is the factor betas
      F is the factor covariance matrix
      S is the idiosyncratic variance matrix

    Parameters
    ----------
    factor_betas : DataFrame
        Factor betas
    factor_cov_matrix : 2 dimensional Ndarray
        Factor covariance matrix
    idiosyncratic_var_matrix : DataFrame
        Idiosyncratic variance matrix
    weights : DataFrame
        Portfolio weights

    Returns
    -------
    predicted_portfolio_risk : float
        Predicted portfolio risk
    """
    assert len(factor_cov_matrix.shape) == 2
    
    #TODO: Implement function
    
    return None


project_tests.test_predict_portfolio_risk(predict_portfolio_risk)

### 查看数据
下面看看如果所有股票的权重都一样，投资组合风险会怎样。

In [ ]:
all_weights = pd.DataFrame(np.repeat(1/len(universe_tickers), len(universe_tickers)), universe_tickers)

predict_portfolio_risk(
    risk_model['factor_betas'],
    risk_model['factor_cov_matrix'],
    risk_model['idiosyncratic_var_matrix'],
    all_weights)

# 创建 Alpha 因子
计算投资组合风险后，开始创建 alpha 因子。在此项目中，我们将创建以下因子：
- 1 年动量因子
- 5 天均值回归行业中性因子
- 5 天均值回归行业中性平滑因子
- 隔夜情感因子
- 隔夜情感平滑因子

## 1 年动量因子
每个因子都有一个假设。对于此因子，假设为“更高的过去 12 个月（252 天）收益率与未来收益率成比例。” 我们根据此假设编写了以下代码：

In [ ]:
from zipline.pipeline.factors import Returns

def momentum_1yr(window_length, universe, sector):
    return Returns(window_length=window_length, mask=universe) \
        .demean(groupby=sector) \
        .rank() \
        .zscore()

## 5 天均值回归行业中性因子
下面根据假设“短期优质股（劣质股）相对于所在行业来说将逆转”实现 `mean_reversion_5day_sector_neutral`。我们使用 `universe` 的收益率数据，根据行业数据去均值、排名，然后转换为 z 分数。

In [ ]:
def mean_reversion_5day_sector_neutral(window_length, universe, sector):
    """
    Generate the mean reversion 5 day sector neutral factor

    Parameters
    ----------
    window_length : int
        Returns window length
    universe : Zipline Filter
        Universe of stocks filter
    sector : Zipline Classifier
        Sector classifier

    Returns
    -------
    factor : Zipline Factor
        Mean reversion 5 day sector neutral factor
    """
    
    #TODO: Implement function
    
    return None


project_tests.test_mean_reversion_5day_sector_neutral(mean_reversion_5day_sector_neutral)

### 查看数据
下面看看一些因子数据的样貌。在计算因子时，我们将查看 2 年的历史数据。

**注意:** _回到 2 年前市场是闭市的。Pipeline 软件包不会处理开始或结束日期是闭市日期的情况。为了解决这个问题，我们再往前推 2 天，将第二天开市日期作为开始日期。_

In [ ]:
factor_start_date = universe_end_date - pd.DateOffset(years=2, days=2)
sector = project_helper.Sector()
window_length = 5

pipeline = Pipeline(screen=universe)
pipeline.add(
    mean_reversion_5day_sector_neutral(window_length, universe, sector),
    'Mean_Reversion_5Day_Sector_Neutral')
engine.run_pipeline(pipeline, factor_start_date, universe_end_date)

## 5 天均值回归行业中性平滑因子
我们根据上个因子的输出创建一个平滑版本。实现 `mean_reversion_5day_sector_neutral_smoothed` 以生成一个 5 天均值回归行业中性平滑因子。调用 `mean_reversion_5day_sector_neutral` 函数以获取非平滑因子，然后使用 `SimpleMovingAverage` 函数使其变平滑。需要再次排名和计算 z 分数。

In [ ]:
from zipline.pipeline.factors import SimpleMovingAverage

def mean_reversion_5day_sector_neutral_smoothed(window_length, universe, sector):
    """
    Generate the mean reversion 5 day sector neutral smoothed factor

    Parameters
    ----------
    window_length : int
        Returns window length
    universe : Zipline Filter
        Universe of stocks filter
    sector : Zipline Classifier
        Sector classifier

    Returns
    -------
    factor : Zipline Factor
        Mean reversion 5 day sector neutral smoothed factor
    """
    
    #TODO: Implement function
    
    return None


project_tests.test_mean_reversion_5day_sector_neutral_smoothed(mean_reversion_5day_sector_neutral_smoothed)

### 查看数据
下面看看一些平滑数据的样貌。

In [ ]:
pipeline = Pipeline(screen=universe)
pipeline.add(
    mean_reversion_5day_sector_neutral_smoothed(5, universe, sector),
    'Mean_Reversion_5Day_Sector_Neutral_Smoothed')
engine.run_pipeline(pipeline, factor_start_date, universe_end_date)

## 隔夜情感因子
对于此因子，我们将使用论文 [_Overnight Returns and Firm-Specific Investor Sentiment_](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2554010) 中的假设。

In [ ]:
from zipline.pipeline.data import USEquityPricing


class CTO(Returns):
    """
    Computes the overnight return, per hypothesis from
    https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2554010
    """
    inputs = [USEquityPricing.open, USEquityPricing.close]
    
    def compute(self, today, assets, out, opens, closes):
        """
        The opens and closes matrix is 2 rows x N assets, with the most recent at the bottom.
        As such, opens[-1] is the most recent open, and closes[0] is the earlier close
        """
        out[:] = (opens[-1] - closes[0]) / closes[0]

        
class TrailingOvernightReturns(Returns):
    """
    Sum of trailing 1m O/N returns
    """
    window_safe = True
    
    def compute(self, today, asset_ids, out, cto):
        out[:] = np.nansum(cto, axis=0)

        
def overnight_sentiment(cto_window_length, trail_overnight_returns_window_length, universe):
    cto_out = CTO(mask=universe, window_length=cto_window_length)
    return TrailingOvernightReturns(inputs=[cto_out], window_length=trail_overnight_returns_window_length) \
        .rank() \
        .zscore()

## 隔夜情感平滑因子
就像之前实现的因子一样，我们也将平滑此因子。

In [ ]:
def overnight_sentiment_smoothed(cto_window_length, trail_overnight_returns_window_length, universe):
    unsmoothed_factor = overnight_sentiment(cto_window_length, trail_overnight_returns_window_length, universe)
    return SimpleMovingAverage(inputs=[unsmoothed_factor], window_length=trail_overnight_returns_window_length) \
        .rank() \
        .zscore()

## 将因子整合到一个管道中
实现了所有因子后，我们将它们添加到一个管道中。

In [ ]:
universe = AverageDollarVolume(window_length=120).top(500)
sector = project_helper.Sector()

pipeline = Pipeline(screen=universe)
pipeline.add(
    momentum_1yr(252, universe, sector),
    'Momentum_1YR')
pipeline.add(
    mean_reversion_5day_sector_neutral(5, universe, sector),
    'Mean_Reversion_5Day_Sector_Neutral')
pipeline.add(
    mean_reversion_5day_sector_neutral_smoothed(5, universe, sector),
    'Mean_Reversion_5Day_Sector_Neutral_Smoothed')
pipeline.add(
    overnight_sentiment(2, 5, universe),
    'Overnight_Sentiment')
pipeline.add(
    overnight_sentiment_smoothed(2, 5, universe),
    'Overnight_Sentiment_Smoothed')
all_factors = engine.run_pipeline(pipeline, factor_start_date, universe_end_date)

all_factors.head()

# 评估 Alpha 因子
*注意：* _我们使用 1 天延迟数据评估 alpha 因子_
## 获取股价数据

In [ ]:
import alphalens as al

assets = all_factors.index.levels[1].values.tolist()
pricing = get_pricing(
    data_portal,
    trading_calendar,
    assets,
    factor_start_date,
    universe_end_date)

## 针对 Alphalens 函数格式化 alpha 因子和股价数据
为了使用各种 alphalens 函数，我们需要对齐索引并将时间转换为 unix 时间戳。我们将在下面的单元格中完成这一步。

In [ ]:
clean_factor_data = {
    factor: al.utils.get_clean_factor_and_forward_returns(factor=factor_data, prices=pricing, periods=[1])
    for factor, factor_data in all_factors.iteritems()}

unixt_factor_data = {
    factor: factor_data.set_index(pd.MultiIndex.from_tuples(
        [(x.timestamp(), y) for x, y in factor_data.index.values],
        names=['date', 'asset']))
    for factor, factor_data in clean_factor_data.items()}

## 量化分析
### 因子收益率
下面看看因子收益率随时间推移的变化情况。它应该基本往右上方变化。

In [ ]:
ls_factor_returns = pd.DataFrame()

for factor, factor_data in clean_factor_data.items():
    ls_factor_returns[factor] = al.performance.factor_returns(factor_data).iloc[:, 0]

(1+ls_factor_returns).cumprod().plot()

### 每个分位数每天的基点
仅仅查看因子加权收益率是不够的。好的 alpha 在数量上也是单调的。我们看看因子收益率的基点。

In [ ]:
qr_factor_returns = pd.DataFrame()

for factor, factor_data in unixt_factor_data.items():
    qr_factor_returns[factor] = al.performance.mean_return_by_quantile(factor_data)[0].iloc[:, 0]

(10000*qr_factor_returns).plot.bar(
    subplots=True,
    sharey=True,
    layout=(4,2),
    figsize=(14, 14),
    legend=False)

你观察到了什么现象？

- 所有的 alpha 都不是**完全单调的**；你可能会疑问为何这样？需要进一步研究和调整 alpha。在所有 alpha（MR 5D 平滑后除外）中，带来最高排名股票的 alpha 为何业绩不是最高？
- 收益率的大部分都来自所有这些 alpha 的**做空部分**。所有 alpha 的第一五分位中的负收益率非常大。这样会导致问题，因为做空股票时，需要找到做空机会；做空的代价很高，甚至没有做空机会。
- 从收益率差异的大小（例如 Q1-Q5）可以看出，在减去交易成本、做空成本等之前，每日收益率约为 0.03%，即 **3 个基点**。假设一年有 252 个交易日，那么年化收益率为 7.56%。交易成本可能会导致收益率减一半。所以，这些 alpha 只适合机构环境，很可能需要应用杠杆效应才能达到理想的收益率。

## 周转率分析

不需要完整的正式回溯测试，我们就能够分析 alpha 随时间推移的稳定情况。这里的稳定性是指每个时段的 alpha 排名变化不大。因为交易有成本，所以在其他一切都不变的情况下，我们希望每个时段的排名变化不大。我们可以使用**因子排名相关系数 (FRA)** 衡量这一点。

[alphalens.performance.factor_rank_autocorrelation](https://quantopian.github.io/alphalens/alphalens.html?highlight=factor_rank_autocorrelation#alphalens.performance.factor_rank_autocorrelation)

In [ ]:
ls_FRA = pd.DataFrame()

for factor, factor_data in unixt_factor_data.items():
    ls_FRA[factor] = al.performance.factor_rank_autocorrelation(factor_data)

ls_FRA.plot(title="Factor Rank Autocorrelation")

## Alpha 的夏普比率

最后我们需要分析因子的夏普比率。请实现 `sharpe_ratio` 以计算因子收益率的夏普比率。

In [ ]:
def sharpe_ratio(factor_returns, annualization_factor):
    """
    Get the sharpe ratio for each factor for the entire period

    Parameters
    ----------
    factor_returns : DataFrame
        Factor returns for each factor and date
    annualization_factor: float
        Annualization Factor

    Returns
    -------
    sharpe_ratio : Pandas Series of floats
        Sharpe ratio
    """
    
    #TODO: Implement function
    
    return None


project_tests.test_sharpe_ratio(sharpe_ratio)

### 查看数据
我们看看因子的夏普比率是多少。通常，夏普比率接近 1.0 或更高对此股票池来说是可接受的单个 alpha。

In [ ]:
daily_annualization_factor = np.sqrt(252)
sharpe_ratio(ls_factor_returns, daily_annualization_factor).round(2)

## 问题：如果我们平滑化动量因子，你认为会发生什么？业绩会上涨、下跌，还是没有大变化?为何？

_#TODO：请在此单元格中填写答案_

**答案：** 

## 组合 Alpha 向量

为了在投资组合中使用这些 alpha，我们需要组合这些 alpha，以便每支股票有一个分数。机器学习非常擅长这个任务。但是在此模块中，我们将采用最简单的组合方法：直接计算每个 alpha 的平均分数。

In [ ]:
selected_factors = all_factors.columns[[1, 2, 4]]
print('Selected Factors: {}'.format(', '.join(selected_factors)))

all_factors['alpha_vector'] = all_factors[selected_factors].mean(axis=1)
alphas = all_factors[['alpha_vector']]
alpha_vector = alphas.loc[all_factors.index.get_level_values(0)[-1]]
alpha_vector.head()

# 由风险模型约束的最优投资组合
我们已经有了 alpha 模型和风险模型。下面创建一个投资组合，它会尽量按照 alpha 模型交易，但是会限制由风险模型衡量的风险。你将为此投资组合构建优化器。为了帮助你完成此任务，我们提供了一个抽象类，叫做 `AbstractOptimalHoldings`。

In [ ]:
from abc import ABC, abstractmethod


class AbstractOptimalHoldings(ABC):    
    @abstractmethod
    def _get_obj(self, weights, alpha_vector):
        """
        Get the objective function

        Parameters
        ----------
        weights : CVXPY Variable
            Portfolio weights
        alpha_vector : DataFrame
            Alpha vector

        Returns
        -------
        objective : CVXPY Objective
            Objective function
        """
        
        raise NotImplementedError()
    
    @abstractmethod
    def _get_constraints(self, weights, factor_betas, risk):
        """
        Get the constraints

        Parameters
        ----------
        weights : CVXPY Variable
            Portfolio weights
        factor_betas : 2 dimensional Ndarray
            Factor betas
        risk: CVXPY Atom
            Predicted variance of the portfolio returns

        Returns
        -------
        constraints : List of CVXPY Constraint
            Constraints
        """
        
        raise NotImplementedError()
        
    def _get_risk(self, weights, factor_betas, alpha_vector_index, factor_cov_matrix, idiosyncratic_var_vector):
        f = factor_betas.loc[alpha_vector_index].values.T * weights
        X = factor_cov_matrix
        S = np.diag(idiosyncratic_var_vector.loc[alpha_vector_index].values.flatten())

        return cvx.quad_form(f, X) + cvx.quad_form(weights, S)
    
    def find(self, alpha_vector, factor_betas, factor_cov_matrix, idiosyncratic_var_vector):
        weights = cvx.Variable(len(alpha_vector))
        risk = self._get_risk(weights, factor_betas, alpha_vector.index, factor_cov_matrix, idiosyncratic_var_vector)
        
        obj = self._get_obj(weights, alpha_vector)
        constraints = self._get_constraints(weights, factor_betas.loc[alpha_vector.index].values, risk)
        
        prob = cvx.Problem(obj, constraints)
        prob.solve(max_iters=500)

        optimal_weights = np.asarray(weights.value).flatten()
        
        return pd.DataFrame(data=optimal_weights, index=alpha_vector.index)

## 目标和约束条件
你将使用此类作为基类并实现 `OptimalHoldings` 类。此类需要实现两个函数： `_get_obj` 和 `_get_constraints` 函数。

`_get_obj` 函数应该返回一个最大化 $ \alpha^T * x \\ $ 的 CVXPY 目标函数，其中 $ x $ 是投资组合权重，$ \alpha $ 是 alpha 向量。

`_get_constraints` 函数应该返回以下约束条件列表：
- $ r \leq risk_{\text{cap}}^2 \\ $
- $ B^T * x \preceq factor_{\text{max}} \\ $
- $ B^T * x \succeq factor_{\text{min}} \\ $
- $ x^T\mathbb{1} = 0 \\ $
- $ \|x\|_1 \leq 1 \\ $
- $ x \succeq weights_{\text{min}} \\ $
- $ x \preceq weights_{\text{max}} $

其中 $ x $ 是投资组合权重，$ B $ 是因子 β，$ r $ 是投资组合风险。

第一个约束条件是预测风险小于某个上限。第二个和第三个约束条件是投资组合因子暴露度上限和下限。第四个约束条件是市场中性约束条件：权重之和必须为零。第五个约束条件是杠杆约束条件：权重的绝对值之和必须小于等于 1.0。最后一个约束条件是单个头寸的下限和上限。

In [ ]:
class OptimalHoldings(AbstractOptimalHoldings):
    def _get_obj(self, weights, alpha_vector):
        """
        Get the objective function

        Parameters
        ----------
        weights : CVXPY Variable
            Portfolio weights
        alpha_vector : DataFrame
            Alpha vector

        Returns
        -------
        objective : CVXPY Objective
            Objective function
        """
        assert(len(alpha_vector.columns) == 1)

        #TODO: Implement function
        
        return None
    
    def _get_constraints(self, weights, factor_betas, risk):
        """
        Get the constraints

        Parameters
        ----------
        weights : CVXPY Variable
            Portfolio weights
        factor_betas : 2 dimensional Ndarray
            Factor betas
        risk: CVXPY Atom
            Predicted variance of the portfolio returns

        Returns
        -------
        constraints : List of CVXPY Constraint
            Constraints
        """
        assert(len(factor_betas.shape) == 2)
        
        #TODO: Implement function
        
        return None

    def __init__(self, risk_cap=0.05, factor_max=10.0, factor_min=-10.0, weights_max=0.55, weights_min=-0.55):
        self.risk_cap=risk_cap
        self.factor_max=factor_max
        self.factor_min=factor_min
        self.weights_max=weights_max
        self.weights_min=weights_min


project_tests.test_optimal_holdings_get_obj(OptimalHoldings)
project_tests.test_optimal_holdings_get_constraints(OptimalHoldings)

### 查看数据
实现 `OptimalHoldings` 类后，我们看看它生成的权重。

In [ ]:
optimal_weights = OptimalHoldings().find(alpha_vector, risk_model['factor_betas'], risk_model['factor_cov_matrix'], risk_model['idiosyncratic_var_vector'])

optimal_weights.plot.bar(legend=None, title='Portfolio % Holdings by Stock')
x_axis = plt.axes().get_xaxis()
x_axis.set_visible(False)

呀，它将大部分的权重分配给了少数几支股票。

In [ ]:
project_helper.get_factor_exposures(risk_model['factor_betas'], optimal_weights).plot.bar(
    title='Portfolio Net Factor Exposures',
    legend=False)

## 通过正则化参数进行优化
为了强制分散投资，我们将在目标函数中使用正则化项。我们将新建一个类 `OptimalHoldingsRegualization` ，它会从 `OptimalHoldings` 类那获取约束条件。在此新类中，请实现 `_get_obj` 函数以返回一个最大化 $ \alpha^T * x + \lambda\|x\|_2\\ $ 的 CVXPY 目标函数，其中 $ x $ 是投资组合权重，$ \alpha $ 是 alpha 向量，$ \lambda $ 是正则化参数。

**注意：**$ \lambda $ 位于 `self.lambda_reg` 中。*

In [ ]:
class OptimalHoldingsRegualization(OptimalHoldings):
    def _get_obj(self, weights, alpha_vector):
        """
        Get the objective function

        Parameters
        ----------
        weights : CVXPY Variable
            Portfolio weights
        alpha_vector : DataFrame
            Alpha vector

        Returns
        -------
        objective : CVXPY Objective
            Objective function
        """
        assert(len(alpha_vector.columns) == 1)
        
        #TODO: Implement function
        
        return None

    def __init__(self, lambda_reg=0.5, risk_cap=0.05, factor_max=10.0, factor_min=-10.0, weights_max=0.55, weights_min=-0.55):
        self.lambda_reg = lambda_reg
        self.risk_cap=risk_cap
        self.factor_max=factor_max
        self.factor_min=factor_min
        self.weights_max=weights_max
        self.weights_min=weights_min
        

project_tests.test_optimal_holdings_regualization_get_obj(OptimalHoldingsRegualization)

### 查看数据

In [ ]:
optimal_weights_1 = OptimalHoldingsRegualization(lambda_reg=5.0).find(alpha_vector, risk_model['factor_betas'], risk_model['factor_cov_matrix'], risk_model['idiosyncratic_var_vector'])

optimal_weights_1.plot.bar(legend=None, title='Portfolio % Holdings by Stock')
x_axis = plt.axes().get_xaxis()
x_axis.set_visible(False)

现在分散效果很不错。

In [ ]:
project_helper.get_factor_exposures(risk_model['factor_betas'], optimal_weights_1).plot.bar(
    title='Portfolio Net Factor Exposures',
    legend=False)

## 通过严格的因子约束条件和目标权重进行优化
另一个常见公式是采用预定义的目标权重 $x^*$（例如分位数投资组合），并获得尽可能接近该投资组合的投资组合，同时满足投资组合级约束条件。对于下个类 `OptimalHoldingsStrictFactor`，你需要实现 `_get_obj` 函数以最小化 $ \|x - x^*\|_2 $，其中 $ x $ 是投资组合权重，$ x^* $ 是目标权重。

In [ ]:
class OptimalHoldingsStrictFactor(OptimalHoldings):
    def _get_obj(self, weights, alpha_vector):
        """
        Get the objective function

        Parameters
        ----------
        weights : CVXPY Variable
            Portfolio weights
        alpha_vector : DataFrame
            Alpha vector

        Returns
        -------
        objective : CVXPY Objective
            Objective function
        """
        assert(len(alpha_vector.columns) == 1)
        
        #TODO: Implement function
        
        return None


project_tests.test_optimal_holdings_strict_factor_get_obj(OptimalHoldingsStrictFactor)

### 查看数据

In [ ]:
optimal_weights_2 = OptimalHoldingsStrictFactor(
    weights_max=0.02,
    weights_min=-0.02,
    risk_cap=0.0015,
    factor_max=0.015,
    factor_min=-0.015).find(alpha_vector, risk_model['factor_betas'], risk_model['factor_cov_matrix'], risk_model['idiosyncratic_var_vector'])

optimal_weights_2.plot.bar(legend=None, title='Portfolio % Holdings by Stock')
x_axis = plt.axes().get_xaxis()
x_axis.set_visible(False)

In [ ]:
project_helper.get_factor_exposures(risk_model['factor_betas'], optimal_weights_2).plot.bar(
    title='Portfolio Net Factor Exposures',
    legend=False)

## 提交项目

完成项目后，就可以提交了。请点击右下角的提交按钮。我们的审阅专家将对项目打分（通过或不通过）并提供反馈。